In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/Sun/ML Shock/Final project/

In [61]:
path = '../'
data_dir = path +'data/'

# path = './' #'/content/drive/MyDrive/Sun/ML Shock/Final project/'
# data_dir = path +'preprocessing/'


data_type = 'eeg_spec'

In [62]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pickle

In [63]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, data_type, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        all_file_list = os.listdir(data_dir)
        self.file_list = [f for f in all_file_list if f.endswith('.npy') and data_type in f]
        self.label_list = [f for f in all_file_list if f.endswith('.npy') and 'y_' in f]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.file_list[idx])
        label_path = os.path.join(self.data_dir, self.label_list[idx])
        data = np.load(file_path)
        class_probs = np.load(label_path)
        label = np.argmax(class_probs)
        if self.transform:
            for trans in self.transform:
                data = trans(data)
        return data, label, class_probs


In [66]:
# # load the scaler
# with open(path + f'files/{data_type}_standard_scaler.pkl', 'rb') as f:
#     standard_scaler = pickle.load(f)

# with open(path + f'files/{data_type}_min_max_scaler.pkl', 'rb') as f:
#     min_max_scaler = pickle.load(f)

In [67]:
# load the data
training_data = CustomDataset(data_dir, data_type = data_type) #, transform = (lambda x: standard_scaler.transform(x),))


In [68]:
train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)

In [69]:
# X_train, y_train, id_train = get_data(get_func= get_eeg_raw_features,
#                                       files = files_train,
#                                       N = int(0.5*len(files_train)),)
# # original_shape = X_train.shape
# X_train = X_train.reshape(X_train.shape[0], -1)
# print('train:', X_train.shape, y_train.shape, id_train.shape)
# # # reshape back to original shape
# # X_train = X_train.reshape(original_shape)

In [70]:
# X_valid, y_valid, id_valid = get_data(get_func= get_eeg_raw_features,
#                                       files = files_valid,
#                                       N = int(0.5*len(files_valid)))
# X_valid = X_valid.reshape(X_valid.shape[0], -1)
# print('valid:', X_valid.shape, y_valid.shape, id_valid.shape)

In [71]:
# X_test, y_test, id_test = get_data(get_func= get_eeg_raw_features,
#                                     files = files_test,
#                                     N = int(0.5*len(files_test)))
# X_test = X_test.reshape(X_test.shape[0], -1)
# print('test:', X_test.shape, y_test.shape, id_test.shape)

## Error function

In [72]:
from scipy.special import kl_div

def get_error(y_true, y_pred):
    return np.mean(kl_div(y_true, y_pred))

In [73]:
error = []
count = 0
dummy = [1/6 for _ in range(6)]
for _, _, y in train_dataloader:
    for i in range(y.shape[0]):
        error.append(get_error(y[i].numpy(), dummy))
        count += 1

print(f'error for uniform predictio: {sum(error)/count:.2f}')

error for uniform predictio: 0.19


## Gradient boost classifier

In [19]:
# # load the pca
# with open(path + f'{data_type}_pca.pkl', 'rb') as f:
#     pca = pickle.load(f)

In [74]:
# load the data
training_data = CustomDataset(data_dir, data_type = data_type)#, 
                            #   transform = (lambda x: standard_scaler.transform(x), lambda x: pca.transform(x.reshape(1,-1))))
train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)

In [75]:
# train a gradient boosting model to predict one of the 6 classes
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
for X, y, _ in train_dataloader:
    model.fit(X.reshape(X.shape[0], -1), y)



In [56]:
# compute the error on the training set
error = 0 
count = 0
for X, _, y in train_dataloader:
    y_pred = model.predict_proba(X.reshape(X.shape[0], -1))
    print(y.shape, y_pred.shape)
    for i in range(y.shape[0]):
        error += get_error(y[i].numpy(), y_pred[i])
        count += 1

print(f'error for the model: {error/count:.2f}')

torch.Size([14, 6]) (14, 2)


ValueError: operands could not be broadcast together with shapes (6,) (2,) 

In [48]:
# # predict the class probabilities
# y_train_pred = model.predict_proba(X_train_pca)
# y_valid_pred = model.predict_proba(X_valid_pca)
# y_test_pred = model.predict_proba(X_test_pca)

# # calculate the error
# error_train = [get_error(y_true, y_pred) for y_true, y_pred in zip(y_train, y_train_pred)]
# error_valid = [get_error(y_true, y_pred) for y_true, y_pred in zip(y_valid, y_valid_pred)]
# error_test = [get_error(y_true, y_pred) for y_true, y_pred in zip(y_test, y_test_pred)]

# print(f'train error: {np.mean(error_train):.2f}')
# print(f'valid error: {np.mean(error_valid):.2f}')
# print(f'test error: {np.mean(error_test):.2f}')